In [ ]:
import os

print(os.path.abspath(os.curdir))

In [ ]:
import os
os.chdir("..")

In [ ]:
print(os.path.abspath(os.curdir))

In [ ]:
import json
import pandas as pd
def build_recap_multitask(model, source_dir, output_dir, setting='overall'):
# model = 'multi-objective'
# source_dir = os.path.join('experiments', model)
# output_dir = os.path.join('experiments', 'recap', model)
# setting='overall'
    recap_overall = pd.DataFrame()
    recap_per_class = pd.DataFrame()
    dataset_dirs = os.listdir(source_dir)
    print(f'datasets: {dataset_dirs}')
    for dataset_dir in dataset_dirs:
        dataset = dataset_dir
        dataset_path = os.path.join(source_dir, dataset_dir)
        encoders = os.listdir(dataset_path)
        print(f'encoders: {encoders}')
        for encoder in encoders:
            encoder_path = os.path.join(dataset_path, encoder)
            class_weights_1 = os.listdir(encoder_path)
            print(f'class_weights_1: {class_weights_1}')
            for weight_1 in class_weights_1:
                weight_1_path = os.path.join(encoder_path, weight_1)
                # losses = os.listdir(weight_1_path)
                class_weights_2 = os.listdir(weight_1_path)
                print(f'class_weights_2: {class_weights_2}')
                for weight_2 in class_weights_2:
                    weight_2_path = os.path.join(weight_1_path, weight_2)
                    losses_1 = os.listdir(weight_2_path)
                    print(f'losses_1: {losses_1}')
                    for loss_1 in losses_1:
                        # if loss_1 != 'bce':
                        #     continue
                        loss_1_path = os.path.join(weight_2_path, loss_1)
                        losses_2 = os.listdir(loss_1_path)
                        print(f'losses_2: {losses_2}')
                        for loss_2 in losses_2:
                            loss_2_path = os.path.join(loss_1_path, loss_2)
                            poolings = os.listdir(loss_2_path)
                            for pooling in poolings:
                                pool_path = os.path.join(loss_2_path, pooling)
                                n_layers = os.listdir(pool_path)
                                for n_layer in n_layers:
                                    layer_path = os.path.join(pool_path, n_layer)
                                    n_heads = os.listdir(layer_path)
                                    for n_head in n_heads:
                                        head_path = os.path.join(layer_path, n_head)
                                        bidirectionals = os.listdir(head_path)
                                        for bidirectional in bidirectionals:
                                            bidirectional_path = os.path.join(head_path, bidirectional)
                                            label_shemas_1 = os.listdir(bidirectional_path)
                                            for label_schema_1 in label_shemas_1:
                                                # if label_schema_1 != 'binary':
                                                #     continue
                                                label_schema_1_path = os.path.join(bidirectional_path, label_schema_1)
                                                label_schemas_2 = os.listdir(label_schema_1_path)
                                                for label_schema_2 in label_schemas_2:
                                                    label_schema_2_path = os.path.join(label_schema_1_path, label_schema_2, '30')
                                                    files = os.listdir(label_schema_2_path)
                                                    eval_report_1_path = os.path.join(label_schema_2_path, files[0])
                                                    eval_report_2_path = os.path.join(label_schema_2_path, files[2])
                                                    scenario_path = os.path.join(label_schema_2_path, files[6])
                                                    with open(eval_report_1_path) as eval_report_1_file:
                                                        eval_report_1 = json.load(eval_report_1_file)
                                                        with open(eval_report_2_path) as eval_report_2_file:
                                                            eval_report_2 = json.load(eval_report_2_file)
                                                            with open(scenario_path) as scenario_json_file:
                                                                scenario = json.load(scenario_json_file)
                                                                if setting == 'overall':
                                                                    accuracy_1 = float(round(eval_report_1['accuracy'] * 100, 5))
                                                                    precision_1 = float(round(eval_report_1['weighted_avg']['precision'] * 100, 5))
                                                                    recall_1 = float(round(eval_report_1['weighted_avg']['recall'] * 100, 5))
                                                                    f1_score_1 = float(round(eval_report_1['weighted_avg']['f1-score'] * 100, 5))
                                                                    accuracy_2 = float(round(eval_report_2['accuracy'] * 100, 5))
                                                                    precision_2 = float(round(eval_report_2['weighted_avg']['precision'] * 100, 5))
                                                                    recall_2 = float(round(eval_report_2['weighted_avg']['recall'] * 100, 5))
                                                                    f1_score_2 = float(round(eval_report_2['weighted_avg']['f1-score'] * 100, 5))
                                                                    best_epoch = int(scenario['best_epoch'])
                                                                    columns = ['dataset', 'encoder', 'n_layer', 'n_head', 'bidirectional', 'pooling', 'weight_1', 'weight_2', 'loss_1', 'loss_2', 'label_schema_1', 'label_schema_2', 'best_epoch', 'accuracy_1', 'precision_1', 'recall_1', 'f1_score_1', 'accuracy_2', 'precision_2', 'recall_2', 'f1_score_2']
                                                                    row_list = [dataset, encoder, n_layer, n_head, bidirectional, pooling, weight_1, weight_2, loss_1, loss_2, label_schema_1, label_schema_2, best_epoch, accuracy_1, precision_1, recall_1, f1_score_1, accuracy_2, precision_2, recall_2, f1_score_2]
                                                                    scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                                                                    recap_overall = pd.concat([recap_overall, scenario_df], ignore_index=True)
                                                                    if not os.path.exists(output_dir):
                                                                        os.makedirs(output_dir)
                                                                    recap_overall.to_excel(os.path.join(output_dir, f'recap_overall_complete_{model}.xlsx'), index=False, float_format='%.5f')
                                                                else:
                                                                    accuracy = float(round(eval_report_1['accuracy'] * 100, 5))
                                                                    best_epoch = int(scenario['best_epoch'])
                                                                    high_score = [float(round(eval_report_1['high'][key] * 100, 5)) for key in eval_report_1['high'].keys()]
                                                                    low_score = [float(round(eval_report_1['low'][key] * 100, 5)) for key in eval_report_1['low'].keys()]
                                                                    medium_score = [float(round(eval_report_1['medium'][key] * 100, 5)) for key in eval_report_1['medium'].keys()]
                                                                    normal_score = [float(round(eval_report_1['normal'][key] * 100, 5)) for key in eval_report_1['normal'].keys()]
                                                                    columns = ['dataset', 'encoder', 'n_layer', 'n_head', 'bidirectional', 'pooling', 'class_weight', 'scenario', 'best_epoch', 'accuracy', 'high_precision', 'high_recall', 'high_f1_score', 'medium_precision', 'medium_recall', 'medium_f1_score', 'low_precision', 'low_recall', 'low_f1_score', 'normal_precision', 'normal_recall', 'normal_f1_score']
                                                                    row_list = [dataset, encoder, n_layer, n_head, bidirectional, pooling, weight, loss, best_epoch, accuracy, high_score[0], high_score[1], high_score[2], medium_score[0], medium_score[1], medium_score[2], low_score[0], low_score[1], low_score[2], normal_score[0], normal_score[1], normal_score[2]]
                                                                    scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                                                                    recap_per_class = pd.concat([recap_per_class, scenario_df], ignore_index=True)
                                                                    recap_per_class.to_excel(os.path.join(output_dir, f'recap_per_class_{model}.xlsx'), index=False, float_format='%.5f')

#### Selected 30 Epochs

In [ ]:
model = 'baseline_15'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

#### Our Own Baseline

In [ ]:
model = 'multi-objective'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'baseline'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')

#### State-of-the-art Baseline

In [ ]:
model = 'transsentlog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'transsentlog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')

In [ ]:
model = 'neurallog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'neurallog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')